In [1]:
#Import python libraries
import pandas as pd
import numpy as np
import os

In [2]:
developed_downloaded = pd.read_csv('developed/developed_downloaded_03_20.csv',encoding = "ISO-8859-1")
developed_scraped = pd.read_csv('developed/developed_scraped_03_20.csv',encoding = "ISO-8859-1")
developed_scraped.drop(['Unnamed: 0'], axis=1, inplace=True)
developed_downloaded.rename(columns={'sub-sector':'subsector'}, inplace=True)
developed_scraped.dropna(subset=['Date'], inplace=True)

In [3]:
#Replace NaN values in capital_investment with 0
developed_scraped['capital_investment'].fillna(0, inplace=True)
developed_downloaded['capital_investment'].fillna(0, inplace=True)
developed_downloaded['jobs_created'].fillna(0, inplace=True)
developed_scraped['jobs_created'].fillna(0, inplace=True)

#Remove alphabets from capital_investment
developed_scraped['capital_investment'] = developed_scraped['capital_investment'].str.replace('[a-zA-Z]', '')

#Replace space with . in capital_investment
developed_scraped['capital_investment'] = developed_scraped['capital_investment'].str.replace(' ', '.')

from decimal import Decimal
#apply function to remove 0 after decimal point
developed_scraped['capital_investment'] = developed_scraped['capital_investment'].apply(lambda x: Decimal(x).normalize())

#Convert object to float with precision 1
developed_scraped['capital_investment'] = developed_scraped['capital_investment'].astype(float).round(1)
developed_downloaded['capital_investment'] = developed_downloaded['capital_investment'].astype(float).round(1)

#Convert object to int
developed_scraped['jobs_created'] = developed_scraped['jobs_created'].astype(int)
developed_downloaded['jobs_created'] = developed_downloaded['jobs_created'].astype(int)

#Remove extra space from sector
developed_scraped['sector'] = developed_scraped['sector'].replace(r'\s+', ' ', regex=True)

#Keep only alphabets in sector
developed_downloaded['sector'] = developed_downloaded['sector'].str.replace('[^a-zA-Z]', ' ')
developed_downloaded['sector'] = developed_downloaded['sector'].replace(r'\s+', ' ', regex=True)

#Remove extra space from subsector
developed_scraped['subsector'] = developed_scraped['subsector'].replace(r'\s+', ' ', regex=True)

#Keep only alphabets in subsector
developed_downloaded['subsector'] = developed_downloaded['subsector'].str.replace('[^a-zA-Z]', ' ')
developed_downloaded['subsector'] = developed_downloaded['subsector'].replace(r'\s+', ' ', regex=True)

#Remove extra space from cluster
developed_scraped['cluster'] = developed_scraped['cluster'].replace(r'\s+', ' ', regex=True)

#Keep only alphabets in cluster column
developed_downloaded['cluster'] = developed_downloaded['cluster'].str.replace('[^a-zA-Z]', ' ')
developed_downloaded['cluster'] = developed_downloaded['cluster'].replace(r'\s+', ' ', regex=True)

#Remove extra space from activity
developed_scraped['activity'] = developed_scraped['activity'].replace(r'\s+', ' ', regex=True)

#Keep only alphabets in activity column
developed_downloaded['activity'] = developed_downloaded['activity'].str.replace('[^a-zA-Z]', ' ')
developed_downloaded['activity'] = developed_downloaded['activity'].replace(r'\s+', ' ', regex=True)

#Replace Co Location with Co-Location in project_type
developed_scraped['project_type'] = developed_scraped['project_type'].str.replace('Co Location', 'Co-Location')

#Keep only numbers in Date column
developed_scraped['year'] = developed_scraped['Date'].str.replace('[^0-9]', '')

#Add 20 suffix to year
developed_scraped['year'] = '20' + developed_scraped['year'].astype(str)

#Convert object to int
developed_scraped['year'] = developed_scraped['year'].astype(int)

#Keep only alphabets in Date column
developed_scraped['month'] = developed_scraped['Date'].str.replace('[^a-zA-Z]', '')

#Convert month to number
developed_scraped['month'] = developed_scraped['month'].replace({'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12})

In [4]:
#Merge developed_downloaded and developed_scraped on jobs_created and capital_investment
developed = pd.merge(developed_downloaded, developed_scraped, on=['jobs_created', 'capital_investment','destination_country','source_country','sector','subsector','cluster','activity','project_type','year','month'], how='inner',suffixes=('', '_y'))
developed.drop(developed.filter(regex='_y$').columns, axis=1, inplace=True)

In [5]:
developed

,project_date,year,month,day,source_region,source_country,source_state,source_adm_region,source_city,destination_region,...,project_status,Date,parent_company,investing_company,Relocation,Freezone,k_sector,k_subsector,k_cluster,k_activity
0,2003-01-01T00:00:00Z,2003,1,1,Western Europe,Germany,Hessen,Not Specified,Wiesbaden,France,...,Opened,Jan-03,Aareal Bank,Mansart,NaN,NaN,financial services,corporate investment banking,financial services,business services
1,2003-01-01T00:00:00Z,2003,1,1,North America,United States,Michigan,Oakland County (MI),Southfield (MI),Germany,...,Opened,Jan-03,AlixPartners,AlixPartners,NaN,NaN,financial services,corporate investment banking,financial services,business services
2,2003-01-01T00:00:00Z,2003,1,1,Western Europe,Germany,Not Specified,Not Specified,Not Specified,Austria,...,Announced,Jan-03,Aqua Montana,Gussinger Mineral,NaN,NaN,food beverages,soft drinks ice,agribusiness,manufacturing
3,2003-01-01T00:00:00Z,2003,1,1,North America,United States,Kentucky,Kenton County (KY),Covington (KY),Netherlands,...,Announced,Jan-03,Ashland,Ashland,NaN,NaN,chemicals,basic chemicals,energy,shared services centre
4,2003-01-01T00:00:00Z,2003,1,1,Western Europe,Germany,Not Specified,Not Specified,Not Specified,United States,...,Announced,Jan-03,Atronic,Atronic Americas,NaN,NaN,consumer products,dolls toy games,consumer goods,headquarters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91433,2020-02-01T00:00:00Z,2020,2,1,North America,United States,Florida,Collier County (FL),Naples (FL),United Kingdom,...,Announced,Feb-20,Arthrex Inc,Arthrex Inc,NaN,NaN,medical devices,medical equipment supplies,life sciences,sales marketing support
91434,2020-02-01T00:00:00Z,2020,2,1,North America,United States,Texas,Collin County (TX),Plano (TX),United Kingdom,...,Opened,Feb-20,Syncapay,daVinci Payments,NaN,NaN,software it services,software publishers except video games,financial services,sales marketing support
91435,2020-02-01T00:00:00Z,2020,2,1,Western Europe,Germany,Nordrhein-Westfalen,Arnsberg,Brilon,United States,...,Announced,Feb-20,Rembe,Rembe Inc,United States,NaN,industrial equipment,measuring control instruments,industrial,headquarters
91436,2020-02-01T00:00:00Z,2020,2,1,Western Europe,France,Ile-de-France,Ile-de-France,Paris,Spain,...,Opened,Feb-20,Rexel,Rexel Spain ABM Rexel,NaN,NaN,electronic components,electrical equipment,ict electronics,sales marketing support


Data above is assembled.</p>
Now we structure as per the Toy Dataset - project-level fdimarkets to prio grid

In [8]:
developed.columns

Index(['project_date', 'year', 'month', 'day', 'source_region',
       'source_country', 'source_state', 'source_adm_region', 'source_city',
       'destination_region', 'destination_country', 'destination_state',
       'destination_adm_region', 'destination_city', 'free_zone', 'relocation',
       'sector', 'subsector', 'cluster', 'activity', 'jobs_created',
       'estimated...20', 'capital_investment', 'estimated...22',
       'project_type', 'project_status', 'Date', 'parent_company',
       'investing_company', 'Relocation', 'Freezone', 'k_sector',
       'k_subsector', 'k_cluster', 'k_activity'],
      dtype='object')

In [13]:
developed['project_date'] = pd.to_datetime(developed[['year', 'month']].assign(DAY=1))

# Combine year and month to create a date column without day
developed['project_date'] = developed['project_date'].dt.strftime('%Y-%m')

In [17]:
developed['project_date']

0        2003-01
1        2003-01
2        2003-01
3        2003-01
4        2003-01
          ...   
91433    2020-02
91434    2020-02
91435    2020-02
91436    2020-02
91437    2020-02
Name: project_date, Length: 91438, dtype: object

In [7]:
developed.to_csv('developed/developed_03_20.csv', index=False)